In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML

sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh = 8
refRatio = 2
c = 1.
CFL = 0.5
nt = 32
RK = 4
deriv = 'U'
order = 2

In [3]:
if (RK == 1):
    TimeIntegratorFunc = ST.ForwardEuler
else:
    if (RK == 2):
        TimeIntegratorFunc = ST.MidpointMeth
    else:
        TimeIntegratorFunc = ST.RK4

if (deriv == 'U'):
    DiffFunc = ST.Upwind
else:
    DiffFunc = TT.CenterDiff#ST.CenterDiff

In [4]:
omega = BT.Grid(nh)
finehalf = list(np.arange(int(nh / refRatio)))# + int(nh / refRatio))
omega.AddPatch(refRatio, finehalf)
degFreed = omega.degFreed
nh_min = omega.nh_min
nh_max = omega.nh_max
t, nt = ST.CalcTime(omega, CFL, c, nt = nt)
derivMat = OT.SpaceDeriv(omega, order, 'deriv')
spatOp = -c * derivMat
strings = omega.strings
timePropOp = LA2.expm(t * spatOp)
omegaF = BT.Grid(nh_max)
wavesF = WT.MakeWaves(omegaF)

[[-0.75  1.25  0.    0.    0.    0.    0.    0.    0.    0.    0.   -0.5 ]
 [-1.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.   -1.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.   -1.    0.    1.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.   -0.5  -0.5   0.    1.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.   -1.    0.    1.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.   -1.    0.    1.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.   -1.25  0.75  0.5   0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.   -1.    0.    1.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.   -1.    0.    1.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.   -1.    0.    1.  ]
 [ 0.5   0.5   0.    0.    0.    0.    0.    0.    0.    0.   -1.    0.  ]]


In [5]:
polyInterp = GTT.GhostCellStencil(2, -0.5)
print(polyInterp)
print(type(polyInterp))

[ 0.5   0.75 -0.25]
<class 'numpy.ndarray'>


In [6]:
def TestPoly(order, x_0, const = 2, tol = 1e-15):
    
    # Create vector of cell bounds.
    bounds = GTT.BoundVals(order, x_0)
    
    # Find intervals of cell bounds.
    h = bounds[:-1] - bounds[1:]
    
    # Create stencil.
    polyInterp = GTT.GhostCellStencil(order, x_0)
    
    # Iterate through monomials up to appropriate order of accuracy to test stencil.
    for k in range(order + 2):
        coefs = np.zeros(k + 1, float)
        coefs[0] = const
        p = np.poly1d(coefs)
        P = np.polyint(p)
        v = (P(bounds[:-1]) - P(bounds[1:])) / h
        print('Order ' + str(k) + ':')

        theor = P(x_0) / x_0
        act = v.transpose() @ polyInterp
        print(theor, act)
        print('')
        if (k < order + 1):
            assert(np.isclose(act, theor, rtol = 0, atol = tol))
    return

In [7]:
blah = TestPoly(5, 0.5)

Order 0:
2.0 1.9999999999999996

Order 1:
0.5 0.5000000000000002

Order 2:
0.16666666666666666 0.16666666666666663

Order 3:
0.0625 0.06250000000000022

Order 4:
0.025 0.0250000000000008

Order 5:
0.010416666666666666 0.010416666666665853

Order 6:
0.004464285714285714 -1.6026785714285705



In [8]:
# Test the stencil works for constant.
c = 2
v = c * np.ones(3, float)
print(v)
out = v.transpose() @ polyInterp
print(out)
tol = 1e-13

assert(np.isclose(c, out, tol))

# Test the stencil works for linear. s is slope.
s = 2
# v is cell centers inputted into linear polynomial.
v = s * np.asarray([-0.5, 0.25, 0.75])
shouldbe = -0.25 * s # -0.25 is the cell center location.
print(v)
out = v.transpose() @ polyInterp
print(out)
assert(np.isclose(out, shouldbe, rtol = 0, atol = tol))

[2. 2. 2.]
2.0
[-1.   0.5  1.5]
-0.49999999999999983


In [9]:
A = np.asarray([[1, -0.5, 1. / 3.], [1, 1. / 4., 1. / 12.], [1, 3. / 4., 7. / 12.]])
print(A)
print('')
Ainv = LA.inv(A)
print(Ainv)

[[ 1.         -0.5         0.33333333]
 [ 1.          0.25        0.08333333]
 [ 1.          0.75        0.58333333]]

[[ 0.16666667  1.08333333 -0.25      ]
 [-1.          0.5         0.5       ]
 [ 1.         -2.5         1.5       ]]


In [57]:
def SpaceDeriv(omega, order, diff):
    
    # Extract info from omega.
    hs = omega.h
    degFreed = omega.degFreed
    
    # Create empty matrizes to fill later.
    blockMat = np.zeros((degFreed, degFreed), float)
    zeroMat = np.zeros(degFreed, float)
    
    # Interpolate ghost cells.
    ghostStencL = GTT.GhostCellStencil(order, -0.5)
    ghostStencR = GTT.GhostCellStencil(order, 0.5)
    
    # Create common constituents of row pieces.
    rightCell = zeroMat + 0
    leftCell = zeroMat + 0
    leftCell[-1] = 1
    ghostL = zeroMat + 0
    ghostR = zeroMat + 0
    ghostL[:order+1] = ghostStencL
    ghostR[:order+1] = ghostStencR
    nrollL = int(np.ceil((order + 1) / 3.))
    nrollR = int(order - nrollL)
    ghostL = np.roll(ghostL, -nrollL)
    ghostR = np.roll(ghostR, -nrollR)
    
    # Define distinct row pieces between upwind and center difference.
    if (diff == 'U'):
        rightCell[0] = 1
        
        cf1v2 = rightCell + 0
        cf1v1 = leftCell + 0
        
        fc1v2 = rightCell + 0
        fc1v1 = leftCell + 0
        
        hMat = OT.StepMatrix(omega)
    else:
        rightCell[1] = 1
        
        cf1v2 = zeroMat + 0
        cf1v2[1] = 0.5
        cf1v2[2] = 0.5
        cf1v1 = leftCell + 0
        
        fc1v2 = ghostR
        fc1v1 = leftCell + 0
        
        hMat = 0.5 * OT.StepMatrix(omega)
    
    # Define common row pieces between upwind and center difference.
    cf2v2 = rightCell + 0
    cf2v1 = ghostL
    
    fc2v2 = rightCell + 0
    fc2v1 = zeroMat + 0
    fc2v1[-1] = 0.5
    fc2v1[-2] = 0.5
    
    # Define rows.
    default = rightCell - leftCell
    cf1 = cf1v2 - cf1v1
    cf2 = cf2v2 - cf2v1
    fc1 = fc1v2 - fc1v1
    fc2 = fc2v2 - fc2v1
    
    # Create vector containing intergrid boundary locations.
    spots = np.roll(hs, -1) - hs
    print('spacing:')
    print(hs)
    print(spots)
    print('')
    
    # Fill matrix.
    for k in range(degFreed):
        if (np.roll(spots, 1 - k)[0] > 0):
            row = fc2
            print('')
            print(k)
            print(row)
            print(np.roll(spots, 1 - k))
            print('')
        else:
            if (np.roll(spots, -k)[0] < 0): # This one's okay.
                row = cf1
                
            else:
                if (np.roll(spots, 1 - k)[0] < 0): # This one's okay.
                    row = cf2 #fc1
                    
                else:
                    if (spots[k] > 0): # This one's okay.
                        row = fc1 #cf2
                    else:
                        row = default
        blockMat[k, :] = np.roll(row, k)
    print(blockMat)
    blockMat = hMat @ blockMat
    return blockMat


In [58]:
derivMat = SpaceDeriv(omega, 1, 'C')

spacing:
[0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.125  0.125  0.125  0.125 ]
[ 0.      0.      0.      0.      0.      0.      0.      0.0625  0.      0.      0.     -0.0625]


8
[ 0.   1.   0.   0.   0.   0.   0.   0.   0.   0.  -0.5 -0.5]
[ 0.0625  0.      0.      0.     -0.0625  0.      0.      0.      0.      0.      0.      0.    ]

[[-0.33333333  1.          0.          0.          0.          0.          0.          0.          0.          0.          0.         -0.66666667]
 [-1.          0.          1.          0.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.         -1.          0.          1.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.         -1.          0.          1.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.         -1.          0. 

In [59]:
omega2 = BT.Grid(nh)
finehalf2 = list(np.arange(int(nh / refRatio)) + int(nh / refRatio))
omega2.AddPatch(refRatio, finehalf2)

In [60]:
derivMat = SpaceDeriv(omega2, 1, 'C')

spacing:
[0.125  0.125  0.125  0.125  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
[ 0.      0.      0.     -0.0625  0.      0.      0.      0.      0.      0.      0.      0.0625]


0
[ 0.   1.   0.   0.   0.   0.   0.   0.   0.   0.  -0.5 -0.5]
[ 0.0625  0.      0.      0.     -0.0625  0.      0.      0.      0.      0.      0.      0.    ]

[[ 0.          1.          0.          0.          0.          0.          0.          0.          0.          0.         -0.5        -0.5       ]
 [-1.          0.          1.          0.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.         -1.          0.          1.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.         -1.          0.          0.5         0.5         0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.         -0.66666667 -0.3

In [48]:
A = np.arange(6)
B = np.roll(A, 1)
print(A)
print(B)

[0 1 2 3 4 5]
[5 0 1 2 3 4]
